In [1]:
from retrievalAugmentedTransformer import RAT, Data_Processing

In [2]:
rat = RAT("AAPL", 22)
dp = Data_Processing(rat)
df = dp.get_data("AAPL", "2020-01-01", "2025-04-02")

X, y = dp.format_data_combined(df, 10, 3)
print(X)
print(y)

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  13 of 13 completed


tensor([[[-1.8734,  0.8852, -0.4331,  ..., -1.6952, -1.4825, -1.0864],
         [-1.8894,  1.0909, -0.4331,  ..., -1.7427, -1.5193, -0.8954],
         [-1.8764,  0.5609, -0.4331,  ..., -1.7091, -1.5074, -0.9163],
         ...,
         [-1.7828,  0.6206, -0.4331,  ..., -1.6786, -1.4883, -1.1049],
         [-1.8062,  1.3874, -0.4331,  ..., -1.6456, -1.4896, -1.0962],
         [-1.8135,  0.6280, -0.4331,  ..., -1.5981, -1.4995, -1.0925]],

        [[-1.8894,  1.0909, -0.4331,  ..., -1.7427, -1.5193, -0.8954],
         [-1.8764,  0.5609, -0.4331,  ..., -1.7091, -1.5074, -0.9163],
         [-1.8841,  0.3804, -0.4331,  ..., -1.7197, -1.5142, -0.9237],
         ...,
         [-1.8062,  1.3874, -0.4331,  ..., -1.6456, -1.4896, -1.0962],
         [-1.8135,  0.6280, -0.4331,  ..., -1.5981, -1.4995, -1.0925],
         [-1.7922,  0.3796, -0.4331,  ..., -1.5692, -1.4610, -1.1049]],

        [[-1.8764,  0.5609, -0.4331,  ..., -1.7091, -1.5074, -0.9163],
         [-1.8841,  0.3804, -0.4331,  ..., -1

In [3]:
rat.train_model(X, y, epochs = 1000)

/opt/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


OperationFailure: PlanExecutor error during aggregation :: caused by :: vector field is indexed with 768 dimensions but queried with 384, full error: {'ok': 0.0, 'errmsg': 'PlanExecutor error during aggregation :: caused by :: vector field is indexed with 768 dimensions but queried with 384', 'code': 8, 'codeName': 'UnknownError', '$clusterTime': {'clusterTime': Timestamp(1743679317, 36), 'signature': {'hash': b']\xaa|k\xa0\xd0\xb4H,\x89\x86\xa9\x18\xccQ\x9dKK\x93K', 'keyId': 7436340156212183079}}, 'operationTime': Timestamp(1743679317, 36)}

In [ ]:
import torch
torch.save(rat.state_dict(), "saved_models/rat.pth")

In [ ]:
import pandas as pd
import requests

def fetch_macro_data():
    # FRED series IDs for macroeconomic indicators
    series_ids = {
        "Fed_Funds_Rate": "FEDFUNDS",
        "CPI": "CPIAUCSL",
        "GDP": "GDPC1",
        "Unemployment_Rate": "UNRATE",
        "Money_Supply": "M2SL",
        "Retail_Sales": "RSXFS",
        "Oil_Price": "IR14250"
    }

    base_url = "https://api.stlouisfed.org/fred/series/observations"
    macro_data = pd.DataFrame()

    # Get the full range of dates you need (e.g., from 2000-01-01 to the most recent available date)
    date_range = pd.date_range(start="2000-01-01", end=pd.to_datetime('today'))

    for column, series_id in series_ids.items():
        # Construct the URL and parameters
        params = {
            "api_key": "d2a5f8b122ea6046576eb67cd258404f",  # Replace with your actual API key
            "file_type": "json",
            "series_id": series_id,
            "observation_start": "2000-01-01",
        }

        # Fetch the data
        response = requests.get(base_url, params=params)
        data = response.json()

        # Parse and format the response data
        observations = data.get("observations", [])
        df = pd.DataFrame(observations)
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)
        df["value"] = pd.to_numeric(df["value"], errors="coerce")

        # Create a DataFrame with all the dates in the date range
        full_df = pd.DataFrame(index=date_range)

        # Merge the fetched data with the full date range
        full_df = full_df.merge(df["value"], how="left", left_index=True, right_index=True)
        
        # Forward fill missing data
        full_df["value"] = full_df["value"].ffill()

        # Add the data to the macro_data DataFrame
        macro_data[column] = full_df["value"]

    # Return the final DataFrame with all macroeconomic indicators
    return macro_data

# Example usage:
macro_data = fetch_macro_data()
#macro_data.to_csv("../data/new_macro_data.csv", index = True)